In [82]:
import supplyactor
import numpy as np

Paramètres 

In [83]:
n = 10
supplyactor.SurStock = 1.001
VarTM = 5


Scénario 1

In [84]:
Magasin = []
for i in range(n):
    Magasin.append(supplyactor.Supermarket(1+VarTM*np.random.random(),"marque"+str(i),i))

for magasin in Magasin : 
    magasin.clients.append(supplyactor.Customer(10*magasin.taille,2*magasin.taille))

In [85]:
for i in range(10*supplyactor.grande_période) : 
    for magasin in Magasin : 
        for client in magasin.clients : 
            client.ARMA()

for i in range(10*supplyactor.grande_période) : 
    for magasin in Magasin : 
        magasin.EnregistrementDemande()
        for client in magasin.clients : 
            client.ARMA()


In [86]:
for magasin in Magasin : 
    magasin.Commande()

In [87]:
for i in range(100*supplyactor.grande_période):
    supplyactor.temps()
    for magasin in Magasin:
        magasin.Inventaire()
        magasin.Vente()
        
        magasin.EnregistrementDemande()
        for client in magasin.clients : 
            client.ARMA()
        if i % supplyactor.grande_période == 0:
            magasin.Inventaire()
         
            magasin.Commande()

        


In [88]:
Gaspillage = 0

for magasin in Magasin : 
    x = magasin.Rebut()
    print(x)
    for product in x.keys() :
        Gaspillage += x[product]
Pénurie = 0
for magasin in Magasin : 
    Pénurie += magasin.quantitéRupture 
Ventes = 0
for magasin in Magasin : 
    Ventes += magasin.ventes

Scénario1 = {"Pénurie" : Pénurie, "Gaspillage":Gaspillage,"Ventes":Ventes}

{'lait': 125.50009697200065, 'oeuf': 59.43428120793865, 'beurre': 284.52272191681584, 'crème': 94.30686370678015, 'fromage': 68.27935496261286, 'ménage': 105.01611624598341, 'aménagement': 68.41364087269264, 'légumes': 84.13611485660763}
{'lait': 125.15157082043255, 'oeuf': 104.47601581534926, 'beurre': 112.51709400739398, 'crème': 71.45198412627575, 'fromage': 203.39432159502954, 'ménage': 85.69332790513582, 'aménagement': 67.0367985286824, 'légumes': 80.01414023764224}
{'lait': 320.47338833694766, 'oeuf': 61.0389731768989, 'beurre': 277.75218993683575, 'crème': 237.07350259162536, 'fromage': 172.17352622204828, 'ménage': 319.9127121175468, 'aménagement': 114.33755199154568, 'légumes': 542.5666093695668}
{'lait': 70.80920357685802, 'oeuf': 100.11939856451907, 'beurre': 184.3036190473017, 'crème': 77.12855027603752, 'fromage': 65.92164195225558, 'ménage': 154.4174545725902, 'aménagement': 205.72760143706594, 'légumes': 52.74779557567095}
{'lait': 117.59808714279383, 'oeuf': 75.02976889

In [89]:
print("Articles en rupture de stock : " + str(Pénurie))
print("Articles jetés : "+ str(Gaspillage))
print("Total des ventes : "+str(Ventes))
print("Gaspillage/commandes : "+  str(100*Gaspillage/(Gaspillage+Ventes)) + " %")
print("Rupture/ventes : "+  str(100*Pénurie/Ventes) +" %")

Articles en rupture de stock : 19891.60861194875
Articles jetés : 11042.107647003675
Total des ventes : 6684588.313925985
Gaspillage/commandes : 0.16491513049206769 %
Rupture/ventes : 0.2975741762661526 %


Scénario 2 : Donnation à une seule organisation 

In [90]:
#instantiation des magasins
Magasin = []
for i in range(n):
    Magasin.append(supplyactor.Supermarket(1+VarTM*np.random.random(),"marque"+str(i),i))

for magasin in Magasin : 
    magasin.clients.append(supplyactor.Customer(10*magasin.taille,2*magasin.taille))
    magasin.BanqueDeDons.append(supplyactor.BanqueAlimentaire(0.1*magasin.taille,0.02*magasin.taille))

In [91]:
#initialisation de la demande des clients et de la collecte des magasins 
for i in range(10*supplyactor.grande_période) : 
    for magasin in Magasin : 
        for client in magasin.clients : 
            client.ARMA()

for i in range(10*supplyactor.grande_période) : 
    for magasin in Magasin : 
        magasin.EnregistrementDemande()
        for client in magasin.clients : 
            client.ARMA()
for magasin in Magasin : 
    magasin.Commande()


In [92]:
#Simulation
for i in range(100*supplyactor.grande_période):
    supplyactor.temps()
    for magasin in Magasin:
        magasin.Inventaire()
        magasin.Vente()
        magasin.Dons()

        magasin.EnregistrementDemande()
        for client in magasin.clients : 
            client.ARMA()
        for banque in magasin.BanqueDeDons : 
            banque.ARMA()
        if i % supplyactor.grande_période == 0:
            magasin.Inventaire()
         
            magasin.Commande()

In [93]:
#Collecte des résultats
Gaspillage = 0

for magasin in Magasin : 
    x = magasin.Rebut()
    print(x)
    for product in x.keys() :
        Gaspillage += x[product]
Pénurie = 0
for magasin in Magasin : 
    Pénurie += magasin.quantitéRupture 
Ventes = 0
for magasin in Magasin : 
    Ventes += magasin.ventes
Dons = 0 
for magasin in Magasin : 
    Dons += magasin.dons

Scénario2 = {"Pénurie" : Pénurie, "Gaspillage":Gaspillage,"Ventes":Ventes, "Dons":Dons}

{'lait': 52.31306971568839, 'oeuf': 30.807237711193434, 'beurre': 42.912593897779765, 'crème': 30.516557772977464, 'fromage': 69.4776829136638, 'ménage': 59.630062563399775, 'aménagement': 71.03889115225692, 'légumes': 33.354474023823805}
{'lait': 64.80494631482264, 'oeuf': 70.50910012601874, 'beurre': 140.13115356969402, 'crème': 39.493740519634876, 'fromage': 15.86217287878802, 'ménage': 58.803352237103155, 'aménagement': 123.48662755003967, 'légumes': 104.16819867008061}
{'lait': 79.02254389572458, 'oeuf': 53.02150320018263, 'beurre': 34.895339114081025, 'crème': 85.95043253529859, 'fromage': 347.6147658665449, 'ménage': 285.94800256354864, 'aménagement': 37.83972800409402, 'légumes': 107.77170318675677}
{'lait': 33.614700465644745, 'oeuf': 91.37975991926375, 'beurre': 39.51659150009304, 'crème': 132.71202535116248, 'fromage': 67.7644901398528, 'ménage': 66.55995978464829, 'aménagement': 128.22211632040936, 'légumes': 160.12288538350293}
{'lait': 170.25584731593557, 'oeuf': 152.4966

In [94]:
#Affichage
print("Articles en rupture de stock : " + str(Pénurie))
print("Articles jetés : "+ str(Gaspillage))
print("Articles données : "+ str(Dons))
print("Total des ventes : "+str(Ventes))
print("Gaspillage/commandes : "+  str(100*Gaspillage/(Gaspillage+Ventes)) + " %")
print("Rupture/ventes : "+  str(100*Pénurie/Ventes) +" %")

Articles en rupture de stock : 20704.14045330932
Articles jetés : 7656.655393898264
Articles données : 2822.6130229940686
Total des ventes : 5203972.080987374
Gaspillage/commandes : 0.14691482799702865 %
Rupture/ventes : 0.3978526427716927 %


Scénario 3 : donnation à un groupe d'association

In [95]:
#instantiation des magasins
Magasin = []
Banques = []
for i in range(n):
    taille = 1+VarTM*np.random.random()
    Magasin.append(supplyactor.Supermarket(taille,"marque"+str(i),i))
    Banques.append(supplyactor.BanqueAlimentaire(0.1*taille,0.02*taille))
    
for magasin in Magasin : 
    magasin.clients.append(supplyactor.Customer(10*magasin.taille,2*magasin.taille))
    magasin.BanqueDeDons = Banques

In [96]:
#initialisation de la demande des clients et de la collecte des magasins 
for i in range(10*supplyactor.grande_période) : 
    for magasin in Magasin : 
        for client in magasin.clients : 
            client.ARMA()

for i in range(10*supplyactor.grande_période) : 
    for banque in Banques : 
        banque.ARMA()
    for magasin in Magasin : 
        magasin.EnregistrementDemande()
        for client in magasin.clients : 
            client.ARMA()
for magasin in Magasin : 
    magasin.Commande()


In [97]:
#Simulation
for i in range(100*supplyactor.grande_période):
    supplyactor.temps()
    for banque in Banques : 
        banque.ARMA()
    for magasin in Magasin:
        magasin.Inventaire()
        magasin.Vente()
        magasin.Dons()

        magasin.EnregistrementDemande()
        for client in magasin.clients : 
            client.ARMA()
        
        if i % supplyactor.grande_période == 0:
            magasin.Inventaire()
         
            magasin.Commande()

In [98]:
#Collecte des résultats
Gaspillage = 0

for magasin in Magasin : 
    x = magasin.Rebut()
    print(x)
    for product in x.keys() :
        Gaspillage += x[product]
Pénurie = 0
for magasin in Magasin : 
    Pénurie += magasin.quantitéRupture 
Ventes = 0
for magasin in Magasin : 
    Ventes += magasin.ventes
Dons = 0 
for magasin in Magasin : 
    Dons += magasin.dons

Scénario3 = {"Pénurie" : Pénurie, "Gaspillage":Gaspillage,"Ventes":Ventes, "Dons":Dons}

{'lait': 0, 'oeuf': 0, 'beurre': 0, 'crème': 0, 'fromage': 0.8033091190828274, 'ménage': 0, 'aménagement': 0, 'légumes': 216.88956611688232}
{'lait': 0, 'oeuf': 0, 'beurre': 0, 'crème': 7.570716804972348, 'fromage': 3.3531897035651586, 'ménage': 11.683238137132363, 'aménagement': 7.44786396159215, 'légumes': 14.119247374324207}
{'lait': 2.06096252232569, 'oeuf': 11.035800714005386, 'beurre': 3.0313280078180167, 'crème': 72.79416056165348, 'fromage': 33.041394285552045, 'ménage': 0.274439012233735, 'aménagement': 58.197817660689616, 'légumes': 99.91711705256887}
{'lait': 27.920341366127676, 'oeuf': 1.609856503750363, 'beurre': 0.5228380232510286, 'crème': 4.020649046990265, 'fromage': 26.42022108568043, 'ménage': 2.6213887865613854, 'aménagement': 30.76080446585806, 'légumes': 35.110202759137394}
{'lait': 29.464216117792287, 'oeuf': 3.455793870542667, 'beurre': 0, 'crème': 30.431499788780723, 'fromage': 88.20259745043352, 'ménage': 4.851544894135294, 'aménagement': 138.7908160462795, 'l

In [99]:
#Affichage
print("Articles en rupture de stock : " + str(Pénurie))
print("Articles jetés : "+ str(Gaspillage))
print("Articles données : "+ str(Dons))
print("Total des ventes : "+str(Ventes))
print("Gaspillage/commandes : "+  str(100*Gaspillage/(Gaspillage+Ventes)) + " %")
print("Rupture/ventes : "+  str(100*Pénurie/Ventes) +" %")

Articles en rupture de stock : 21261.809136405132
Articles jetés : 3267.1135101009822
Articles données : 7443.671622656777
Total des ventes : 6261884.543909831
Gaspillage/commandes : 0.05214739704236339 %
Rupture/ventes : 0.3395432954298701 %


Comparaison des résulats

In [100]:
print("amélioration dans le scénario2")
print("Gaspillage réduit d'un facteur : ",Scénario1["Gaspillage"]/Scénario2["Gaspillage"])
print("Rupture réduit d'un facteur : ",Scénario1["Pénurie"]/Scénario2["Pénurie"])
print("Ventes augmentée d'un facteur",Scénario1["Ventes"]/Scénario2["Ventes"])
print("amélioration dans le scénario3")
print("Gaspillage réduit d'un facteur : ",Scénario1["Gaspillage"]/Scénario3["Gaspillage"])
print("Rupture réduit d'un facteur : ",Scénario1["Pénurie"]/Scénario3["Pénurie"])
print("Ventes augmentée d'un facteur",Scénario1["Ventes"]/Scénario3["Ventes"])

amélioration dans le scénario2
Gaspillage réduit d'un facteur :  1.442158106763345
Rupture réduit d'un facteur :  0.9607551038792005
Ventes augmentée d'un facteur 1.2845165596387456
amélioration dans le scénario3
Gaspillage réduit d'un facteur :  3.379774719447191
Rupture réduit d'un facteur :  0.9355557885189422
Ventes augmentée d'un facteur 1.0675042420619631
